In [ ]:
import os
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
#Train Data
meta_data=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
print(len(meta_data))
meta_data.head()

In [ ]:
#Test data
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
#Lets check distribution of target variable
plt.hist(meta_data['Pawpularity'])

We can see the distribution is normal with some outliners

In [ ]:
#Checking inter dependencies betweeen variables(Correlation matrix)
#TRAIN DATA
corr = meta_data.drop('Id',axis=1).corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

In [ ]:
#TEST DATA
corr = test.drop('Id',axis=1).corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)

There is lot more correlation in test data wrt train data

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(meta_data.drop(['Pawpularity','Id'],axis=1),meta_data.Pawpularity)

In [ ]:
def reg_score(real,pred):    #Uses it to check deviation of predicted output with a +2,-1 error
    count=0
    for r,p in zip(real,pred):
        if r-2<=p<=r+2:count+=1
    return count/len(real)

Now some regression models:-
Always start with linear regression
* Linear Reg
* XG Boost
* Gradient Boost

In [ ]:
#M1
linReg=LinearRegression().fit(x_train,y_train)
linReg_predict=linReg.predict(x_test)
print(linReg.score(x_test,y_test))
print(reg_score(y_test,linReg_predict))
mse = mean_squared_error(y_test,linReg_predict, squared= False)
print(mse)

In [ ]:
#M2
xgb = XGBRegressor(n_estimators=1000, max_depth=6, eta=0.1, subsample=0.7, colsample_bytree=0.8)
xgb.fit(x_train,y_train)
xgb_predict=xgb.predict(x_test)
print(xgb.score(x_test,y_test))
print(reg_score(y_test,xgb_predict))
mse = mean_squared_error(y_test,xgb_predict, squared= False)
print(mse)

In [ ]:
#M3
gbr = GradientBoostingRegressor()
gbr.fit(x_train, y_train)
gb_predict = gbr.predict(x_test)
mse = mean_squared_error(y_test,gb_predict, squared= False)
print(mse)

In [ ]:
submit = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
submit['Pawpularity'] = linReg_predict=linReg.predict(test.drop(['Id'],axis=1))
submit.head(10)

In [ ]:
submit.to_csv('submission.csv',index=False)